In [72]:
import sys
sys.path.append('/home/s94zalek/shape_matching')

# from datasets_code import build_dataloader, build_dataset
# from utils.options import parse_options
# from train import create_train_val_dataloader
import torch
import numpy as np

import os
os.chdir('/home/s94zalek/shape_matching')

# print current working directory
print(os.getcwd())

/home/s94zalek/shape_matching


In [73]:
import datasets_code.shape_dataset as shape_dataset

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [81]:
dataset_faust_single = shape_dataset.SingleFaustDataset(
    data_root='data/FAUST_r',
    phase='train',
    return_faces=True,
    return_evecs=True, num_evecs=32,
    return_corr=True, return_dist=False
)
dataset_faust_pair = shape_dataset.PairFaustDataset(
    data_root='data/FAUST_r',
    phase='train',
    return_faces=True,
    return_evecs=True, num_evecs=32,
    return_corr=True, return_dist=False
)


Calculating functional maps: 100%|███████████████████████████████████████████████████████████████████████████| 6400/6400 [00:15<00:00, 425.69it/s]


In [80]:
import torch
import yaml
import torch

from my_code.models.diag_conditional import DiagConditionedUnet

exp_dir = '/home/s94zalek/shape_matching/my_code/experiments/test_32'

with open(exp_dir + '/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

model = DiagConditionedUnet(config["model_params"]).to('cuda')

# load checkpoint_29.pt
model.load_state_dict(torch.load(exp_dir + '/checkpoints/checkpoint_29.pt'))
model = model.to('cuda')

In [88]:
class DatasetFromListOfDicts(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __getitem__(self, index):
        return self.data_list[index]

    def __len__(self):
        return len(self.data_list)
    

subset_indices = list(range(0, 80))

data_evals = []
for i in subset_indices:
    data_evals.append({
        'first': {},
        'second': {
            'C_gt_xy': dataset_faust_pair[i]['second']['C_gt_xy'].unsqueeze(0), 
            'evals': dataset_faust_pair[i]['second']['evals'].unsqueeze(0)  
        }
    })
    
test_subset = DatasetFromListOfDicts(data_evals)

In [89]:
from tqdm import tqdm
from diffusers import DDPMScheduler
import my_code.diffusion_training.sample_model as sample_model

test_loader = torch.utils.data.DataLoader(
    test_subset,
    batch_size=len(test_subset), shuffle=False
    )
noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2',
                                clip_sample=True)

x_sampled = sample_model.sample(model, test_loader, noise_scheduler)

Sampling test_loader...:   0%|          | 0/1 [00:00<?, ?it/s]

Denoising...:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 4, figsize=(12, 5))

ax_0 = ax[0].imshow(x_sampled[0, 0].cpu().numpy(), cmap='Greys')
# plt.colorbar(ax_0, ax=ax[0])
ax_1 = ax[1].imshow(x_sampled[1, 0].cpu().numpy(), cmap='Greys')
# plt.colorbar(ax_1, ax=ax[1])
ax_2 = ax[2].imshow(x_sampled[2, 0].cpu().numpy(), cmap='Greys')
# plt.colorbar(ax_2, ax=ax[2])
ax_3 = ax[3].imshow(x_sampled[3, 0].cpu().numpy(), cmap='Greys')
# plt.colorbar(ax_3, ax=ax[3])

plt.show()

In [ ]:
import trimesh

scene = trimesh.Scene()

template_mesh = trimesh.load('/home/s94zalek/shape_matching/data/SURREAL_full/template/template.ply')
# add color
template_mesh.visual.vertex_colors = np.array([0, 255, 0, 255], dtype=np.uint8)

# normalize to unit sphere
# template_mesh.vertices = template_mesh.vertices - np.mean(template_mesh.vertices, axis=0)
# template_mesh.vertices = template_mesh.vertices / np.max(np.abs(template_mesh.vertices))

scene.add_geometry(template_mesh)


mesh_0 = trimesh.Trimesh(
    vertices=dataset_faust_pair[subset_indices[0]]['second']['verts'].cpu().numpy() +\
        np.array([1, 0, 0]),
    faces=dataset_faust_pair[subset_indices[0]]['second']['faces'].cpu().numpy(),
    vertex_colors=np.array([255, 0, 0, 255], dtype=np.uint8))
scene.add_geometry(mesh_0)

mesh_scape = trimesh.Trimesh(
    vertices=dataset_scape_pair[1]['second']['verts'],
    faces=dataset_scape_pair[1]['second']['faces'],
    vertex_colors=np.array([0, 0, 255, 255], dtype=np.uint8))

# rotate the mesh by 90 degrees along y axis
mesh_scape.apply_transform(trimesh.transformations.rotation_matrix(-np.pi/2, [0, 1, 0]))
mesh_scape.vertices += np.array([2, 0, 0])
scene.add_geometry(mesh_scape)

    
# add axis
axis = trimesh.creation.axis(axis_length=1)
scene.add_geometry(axis)

scene.show()

In [120]:
template_mesh.vertices.shape, mesh_0.vertices.shape

((6890, 3), (4999, 3))

In [ ]:
# align template mesh and mesh_0 with procrustes

from scipy.spatial.transform import Rotation as R

random_vertices_template = np.random.choice(template_mesh.vertices.shape[0], 4000, replace=False)
random_vertices_mesh_0 = np.random.choice(mesh_0.vertices.shape[0], 4000, replace=False)

# align template mesh and mesh_0 with procrustes
transform_matrix, template_transformed, cost = trimesh.registration.procrustes(
    template_mesh.vertices[random_vertices_template],
    mesh_0.vertices[random_vertices_mesh_0],
    reflection=False
    )

# apply 4x4 transformation matrix to template mesh
template_vertices_transformed = trimesh.transform_points(template_mesh.vertices, transform_matrix)

mesh_template_aligned = trimesh.Trimesh(
    vertices = template_vertices_transformed,
    faces = template_mesh.faces,
    vertex_colors = np.array([0, 255, 0, 255], dtype=np.uint8)
)

scene_aligned = trimesh.Scene()
scene_aligned.add_geometry(mesh_template_aligned)
scene_aligned.add_geometry(mesh_0)

# add axis
axis = trimesh.creation.axis(axis_length=1)
scene_aligned.add_geometry(axis)

scene_aligned.show()

In [ ]:
def center(input_mesh):
    """
    This function center the input mesh using it's bounding box
    Input : mesh
    output : centered mesh and translation vector
    """
    bbox = np.array([[np.max(input_mesh.vertices[:,0]), np.max(input_mesh.vertices[:,1]), np.max(input_mesh.vertices[:,2])], [np.min(input_mesh.vertices[:,0]), np.min(input_mesh.vertices[:,1]), np.min(input_mesh.vertices[:,2])]])

    translation = (bbox[0] + bbox[1]) / 2
    points = input_mesh.vertices - translation
    mesh = trimesh.Trimesh(vertices=points, faces=input_mesh.faces, process= False)
    return mesh, translation

def scale(input_mesh, mesh_ref):
    """
    This function scales the input mesh to have the same volume as a reference mesh Intended to be used before a feed forward pass in pointNet
    Input : file path
    mesh_ref : reference mesh path
    output : scaled mesh
    """
    area = np.power(mesh_ref.volume / input_mesh.volume, 1.0/3)
    mesh= trimesh.Trimesh( vertices =  input_mesh.vertices * area, faces= input_mesh.faces, process = False)
    return mesh, area


temlplate_centered, translation_template = center(template_mesh)
mesh_0_centered, translation_mesh_0 = center(mesh_0)
scape_centered, translation_scape = center(mesh_scape)

template_scaled, scale_template = scale(temlplate_centered, mesh_0_centered)
scape_scaled, scale_scape = scale(scape_centered, mesh_0_centered)

scene.geometry.clear()

template_scaled.visual.vertex_colors = np.array([0, 255, 0, 255], dtype=np.uint8)
mesh_0_centered.visual.vertex_colors = np.array([255, 0, 0, 255], dtype=np.uint8)
scape_scaled.visual.vertex_colors = np.array([0, 0, 255, 255], dtype=np.uint8)

scene.add_geometry(template_scaled)
scene.add_geometry(mesh_0_centered)
scene.add_geometry(scape_scaled)

# add axis
scene.add_geometry(axis)

scene.show()

In [183]:
from utils.fmap_util import nn_query

p2p_template = nn_query(
    torch.tensor(template_scaled.vertices),
    torch.tensor(mesh_0_centered.vertices)
    )
p2p_mesh_0 = nn_query(
    torch.tensor(mesh_0_centered.vertices),
    torch.tensor(template_scaled.vertices)
    )

In [185]:
p2p_template, p2p_mesh_0, dataset_faust_pair[subset_indices[0]]['second']['corr']

(tensor([   1,    0,  354,  ..., 4810, 3510, 3484]),
 tensor([   1,   71,   70,  ..., 4907, 4907, 4907]),
 tensor([4414, 1129, 1929,  ..., 3617,  173, 2482]))

In [193]:
mesh_0_colored.vertices.shape

(4999, 3)

In [ ]:
# add color map to template mesh

scene.geometry.clear()

# make a color map based on y coordinate
cmap = trimesh.visual.color.interpolate(template_scaled.vertices[:, 1], 'jet')

colored_template = trimesh.Trimesh(vertices=template_scaled.vertices + np.array([1, 0, 0]), faces=template_scaled.faces, vertex_colors=cmap)
# add the first mesh
scene.add_geometry(colored_template)

# mesh_0_centered.visual.vertex_colors = cmap[p2p_template]
# mesh_0_centered.visual.vertex_colors = cmap[dataset_faust_pair[subset_indices[0]]['second']['corr']]

mesh_0_colored = trimesh.Trimesh(
    vertices=mesh_0_centered.vertices,
    faces=mesh_0_centered.faces,
    vertex_colors=cmap[p2p_template[:len(mesh_0_centered.vertices)]]
)

# mesh_0_centered.visual.vertex_colors = cmap[]
scene.add_geometry(mesh_0_colored)

scene.show()

In [205]:
dataset_corr = dataset_faust_pair[subset_indices[0]]['second']['corr']
# pad with zeros to match n of vertices on template mesh
dataset_corr = torch.cat([dataset_corr, torch.zeros(template_scaled.vertices.shape[0] - dataset_corr.shape[0])]).int()

In [204]:
dataset_corr

tensor([4414., 1129., 1929.,  ...,    0.,    0.,    0.])

In [ ]:
# add color map to template mesh

scene.geometry.clear()

# make a color map based on y coordinate
cmap = trimesh.visual.color.interpolate(mesh_0_centered.vertices[:, 1], 'jet')

mesh_0_colored = trimesh.Trimesh(
    vertices=mesh_0_centered.vertices,
    faces=mesh_0_centered.faces,
    vertex_colors=cmap
)
scene.add_geometry(mesh_0_colored)


colored_template = trimesh.Trimesh(
    vertices=template_scaled.vertices + np.array([1, 0, 0]),
    faces=template_scaled.faces,
    vertex_colors=cmap[dataset_corr]
    )
# add the first mesh
scene.add_geometry(colored_template)

# mesh_0_centered.visual.vertex_colors = cmap[p2p_template]
# mesh_0_centered.visual.vertex_colors = cmap[dataset_faust_pair[subset_indices[0]]['second']['corr']]


scene.show()

In [180]:
dataset_scape_pair = shape_dataset.PairScapeDataset(
    data_root='data/SCAPE_r',
    phase='train',
    return_faces=True,
    return_evecs=True, num_evecs=32,
    return_corr=True, return_dist=False
)


Calculating functional maps: 100%|███████████████████████████████████████████████████████████████████████████| 2601/2601 [00:06<00:00, 431.50it/s]


In [ ]:
scene.geometry.clear()

for i in range(5):
    mesh_scape = trimesh.Trimesh(
        vertices=dataset_scape_pair[i]['second']['verts'].cpu().numpy() + np.array([i, 0, 0]),
        faces=dataset_scape_pair[i]['second']['faces'].cpu().numpy(),
        # vertex_colors=np.array([255, 0, 0, 255], dtype=np.uint8))
    )
    scene.add_geometry(mesh_scape)
    
scene.show()

In [235]:
data_scape_6 = dataset_scape_pair[6]['second']
data_faust_0 = dataset_faust_pair[3]['second']
# data_faust_0 = dataset_scape_pair[7]['second']

In [236]:
import utils.fmap_util as fmap_util

Cxy_est = torch.linalg.lstsq(
    data_faust_0['evecs'][data_faust_0['corr']],
    data_scape_6['evecs'][data_scape_6['corr']]
    ).solution
p2p_est = fmap_util.fmap2pointmap(
    Cxy_est,
    evecs_x=data_scape_6['evecs'],
    evecs_y=data_faust_0['evecs'],
    )

p2p_est.shape

torch.Size([5001])

In [227]:
data_scape_6['verts'].shape, data_faust_0['verts'].shape

(torch.Size([5000, 3]), torch.Size([4999, 3]))

In [ ]:
# add color map to template mesh

scene.geometry.clear()

# make a color map based on y coordinate
cmap = trimesh.visual.color.interpolate(data_scape_6['verts'][:, 1], 'jet')

mesh_scape = trimesh.Trimesh(
    vertices=data_scape_6['verts'],
    faces=data_scape_6['faces'],
    vertex_colors=cmap
)
scene.add_geometry(mesh_scape)

mesh_faust = trimesh.Trimesh(
    vertices=data_faust_0['verts'] + np.array([1, 0, 0]),
    faces=data_faust_0['faces'],
    vertex_colors=cmap[p2p_est][:len(data_faust_0['verts'])]
)
scene.add_geometry(mesh_faust)

scene.show()

In [ ]:
# open images 0 - 10 and show them in a grid
# /home/s94zalek/shape_matching/data/FAUST_r/mesh_images/train/tr_reg_000.png

import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 5, figsize=(10, 5))

for i in range(10):
    img = plt.imread(f'/home/s94zalek/shape_matching/data/FAUST_r/mesh_images/train/tr_reg_{i:03d}.png')
    ax[i // 5, i % 5].imshow(img)
    # ax[i // 5, i % 5].axis('off')
    
plt.show()

In [ ]:
# now show 0 - 10 from here
# /home/s94zalek/shape_matching/data/SCAPE_r/mesh_images/train/mesh000.png

fig, ax = plt.subplots(2, 5, figsize=(10, 5))

for i in range(10):
    img = plt.imread(f'/home/s94zalek/shape_matching/data/SCAPE_r/mesh_images/train/mesh{i:03d}.png')
    ax[i // 5, i % 5].imshow(img)
    # ax[i // 5, i % 5].axis('off')
    
plt.show()